In [ ]:
import sys
import json
import subprocess
import numpy as np
from subprocess import PIPE
import tensorflow as tf
from PIL import Image
import pickle

sys.path.append("./cpu")
import IPUnit_CPU
# !pip3 install networkx

In [ ]:
# define configs
# compiler = NC4HLS.Compiler()
# compiler.model_file = "./test_model/keras_mnist_DCAE.h5"
# compiler.weight_file = "./test_model/keras_mnist_DCAE_weights.h5"
# compiler.export_pynq_code_path = "./pynq/"
# compiler.export_hls_code_path = "./hls/"
export_cpu_code_path = "./cpu/"
# compiler.log_path = "./log/"
# compiler.src_code_path = "../src"
# compiler.thread = 7
# compiler.show_graph = False
# compiler.print_debug = True

json_file = export_cpu_code_path + "Config.json"
with open(json_file, "r") as jf:
    ip_config = json.load(jf)

# data_types_dict = {"data_type": "q4.12",
#                    "addr_type": "int16_t",
#                    "signal_type": "int16_t"}
data_types_dict = ip_config["data_types"]
thread_num = ip_config["thread_num"]

data_types_dataset = np.float32
tf.keras.backend.set_floatx('float32')
data_types = data_types_dict

# generate a SharedObject
command = "cd " + export_cpu_code_path + " && ./GenerateSharedObject.sh\n cd ./.."
proc = subprocess.run(command, shell=True, stdout=PIPE, stderr=PIPE, text=True)
print("stdout : {}".format(proc.stdout))
print("stderr : {}".format(proc.stderr))

src_size, dst_size = (1, 28, 28, 1), (1, 28, 28, 1)

fake_data_src = np.array(Image.open('./keras_mnist_DCAE_input_2D.png'), dtype=data_types_dataset)
fake_data_src = (fake_data_src - np.min(fake_data_src)) / (np.max(fake_data_src) - np.min(fake_data_src))
fake_data_src = [fake_data_src.reshape(src_size)]

data_vec_config = ip_config["data_vec_config"]
_src_data = [[0 for x in range(y)] for y in data_vec_config.values()]
for bank, i in enumerate(fake_data_src):
    _i = i.reshape(-1).tolist()
    for j, k in enumerate(_i):
        _src_data[bank][j] = k
opcode = ip_config["opcode"]

control_code_file = export_cpu_code_path + "control_code.bin"
weights_vec_file = export_cpu_code_path + "Weights.bin"
shared_object = export_cpu_code_path + "CoreUnit_CPU.so"
ipunit_output = IPUnit_CPU.IPUnit_CPU(_src_data,
                                      opcode,
                                      control_code_file=control_code_file,
                                      weights_vec_file=weights_vec_file,
                                      shared_object=shared_object,
                                      data_types=data_types_dict,
                                      print_debug=False)

In [ ]:

def export_as_png(data: list, shape, file_name: str):
    _output = np.array(data).reshape(shape)
    _output = (_output - np.min(_output)) / (np.max(_output) - np.min(_output)) * 255
    _output = _output.astype(np.uint8)
    _img = Image.fromarray(_output)
    _img.save(file_name)

_dst_size = dst_size[0] * dst_size[1] * dst_size[2] * dst_size[3]
ipunit_output[0] = ipunit_output[0][0:_dst_size]

export_as_png(ipunit_output[0], dst_size[1:3], "./result_img/ipunit_output_rpi.png")
Image.open("./result_img/ipunit_output_rpi.png")

In [ ]:
output_bin_files = ["./result_img/rpi_output.bin"]
output_bin_data = [ipunit_output[0]]
for serialized_item in zip(output_bin_files, output_bin_data):
    with open(serialized_item[0], "wb") as f:
        pickle.dump(serialized_item[1], f)

In [ ]:
src_data_name = ["./test_img/src_data_{:03}.png".format(i) for i in range(100)]
execute_time = []

for cnt, src_img in enumerate(src_data_name):
    # fake_data_src = np.array(Image.open('./../test_img/keras_mnist_DCAE_input_2D.png'), dtype=data_types_dataset)
    fake_data_src = np.array(Image.open(src_img), dtype=data_types_dataset)
    fake_data_src = (fake_data_src - np.min(fake_data_src)) / (np.max(fake_data_src) - np.min(fake_data_src))
    fake_data_src = [fake_data_src.reshape(src_size)]

    # run the IPUnit
    print("\n\n\n----Run IPUnit---\n\n\n")

    data_vec_config = ip_config["data_vec_config"]
    _src_data = [[0 for x in range(y)] for y in data_vec_config.values()]
    for bank, i in enumerate(fake_data_src):
        _i = i.reshape(-1).tolist()
        for j, k in enumerate(_i):
            _src_data[bank][j] = k
    opcode = ip_config["opcode"]

    control_code_file = export_cpu_code_path + "control_code.bin"
    weights_vec_file = export_cpu_code_path + "Weights.bin"
    shared_object = export_cpu_code_path + "CoreUnit_CPU.so"
    ipunit_output, _execute_time = IPUnit_CPU.IPUnit_CPU_execute_time(_src_data,
                                                                     opcode,
                                                                     control_code_file=control_code_file,
                                                                     weights_vec_file=weights_vec_file,
                                                                     shared_object=shared_object,
                                                                     data_types=data_types_dict,
                                                                     print_debug=False)
    execute_time.append(_execute_time)

In [ ]:
run_time = []
for i, single_record in enumerate(execute_time):
    for j, kv in enumerate(single_record.items()):
        k, v = kv
        if j == 0:
            fields = list(v.keys())
            single_run_time = [0.0 for x in range(len(v))]
        _values = list(v.values())
        print(_values)
        for k, _time in enumerate(_values):
            single_run_time[k] += _time
    run_time.append(single_run_time)

print()
print(fields)
print(run_time)
print("total_time(average) : {}[ms]".format(sum(map(sum, run_time)) / len(run_time)))
with open("./result_img/result_time_rpi.time", "wb") as f:
    pickle.dump({"field": fields, "run_time": run_time, "execute_time": execute_time, "device": "rpi4_4gb"}, f)

In [ ]:
run_core = []
for i in run_time:
    run_core.append(i[-2])
print("run_core_average : {} \pm {}[ms]".format(np.mean(run_core), np.std(run_core)))